In [ ]:
# !pip3 install torch
# !pip3 install datasets

In [1]:
import json
import random
import torch

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("causalnlp/CLadder")

In [2]:
import pandas as pd

dataset = pd.read_csv("data_full_v1.csv")

In [3]:
# dataset = dataset[0:100]

In [4]:
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)

dataset['prompt'] = dataset.apply(lambda row: (row['prompt']).removeprefix('Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: '), axis=1)


In [5]:
dataset['prompt'] = dataset['prompt'].str.lower()

In [6]:
#remove this

dataset

,id,prompt,label,ground_truth
0,4,husband has a direct effect on wife and alarm ...,yes,1
1,7,husband has a direct effect on wife and alarm ...,no,0
2,8,husband has a direct effect on wife and alarm ...,yes,1
3,15,husband has a direct effect on wife and alarm ...,no,0
4,21,husband has a direct effect on wife and alarm ...,no,0
...,...,...,...,...
10107,31012,zuph has a direct effect on glimx. jyka has a ...,yes,1
10108,31014,zuph has a direct effect on glimx. jyka has a ...,yes,1
10109,31015,zuph has a direct effect on glimx. jyka has a ...,no,0
10110,31016,zuph has a direct effect on glimx. jyka has a ...,no,0


In [ ]:
# dataset = dataset['train'].to_pandas()
# dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['ground_truth']=='yes' else 0, axis=1)

In [7]:
dataset = json.loads(dataset.to_json(orient ='records'))

# Shuffle the dataset
random.shuffle(dataset)

# Define split sizes for a 75-25 split
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size

# Split the dataset
train_data = dataset[:train_size]
validation_data = dataset[train_size:]

In [8]:
from torch.utils.data import Dataset

class CladderDataset(Dataset):
    def __init__(self, cladder_data, tokenizer):
        self.cladder_data = cladder_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.cladder_data)

    def __getitem__(self, idx):
        data = self.cladder_data[idx]
        # print(data)
        prompt = data['prompt']
        # hypotheses = [data['hypothesis1'], data['hypothesis2']]
        label = data['ground_truth']

        # Tokenize
        encoded_input = self.tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').to("cuda")
        # print(encoded_input)
        encoded_input['labels'] = torch.tensor(label).to("cuda")

        # print(encoded_input)

        return encoded_input


In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import BertTokenizer as tkz
from transformers import BertForSequenceClassification as mpc

# Load tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

# run = 0
# for run in range(1):

for run in range(3):
  model_base = 'bert-base-uncased'
  tokenizer = tkz.from_pretrained(model_base)
  model = mpc.from_pretrained(model_base, num_labels = 2)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  # Prepare the DataLoader
  cladder_dataset = CladderDataset(train_data, tokenizer)
  dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

  # Optimizer and Loss Function
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  loss_fn = torch.nn.CrossEntropyLoss()



  # Training Loop
  model.train()

  #change to 3

  for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  torch.save(model.state_dict(), model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch: 0, Loss: 0.787660539150238
Epoch: 0, Loss: 0.6218639612197876
Epoch: 0, Loss: 0.7497864961624146
Epoch: 0, Loss: 0.6763140559196472
Epoch: 0, Loss: 0.704136848449707
Epoch: 0, Loss: 0.7142633199691772
Epoch: 0, Loss: 0.6721822619438171
Epoch: 0, Loss: 0.6730597615242004
Epoch: 0, Loss: 0.6547747254371643
Epoch: 0, Loss: 0.6337489485740662
Epoch: 0, Loss: 0.687499463558197
Epoch: 0, Loss: 0.6732804179191589
Epoch: 0, Loss: 0.7909969091415405
Epoch: 0, Loss: 0.7069888710975647
Epoch: 0, Loss: 0.7228716015815735
Epoch: 0, Loss: 0.6744825839996338
Epoch: 0, Loss: 0.6680995225906372
Epoch: 0, Loss: 0.6815815567970276
Epoch: 0, Loss: 0.5854995250701904
Epoch: 0, Loss: 0.7304427623748779
Epoch: 0, Loss: 0.7835432887077332
Epoch: 0, Loss: 0.670037567615509
Epoch: 0, Loss: 0.7450265884399414
Epoch: 0, Loss: 0.6833890080451965
Epoch: 0, Loss: 0.6391577124595642
Epoch: 0, Loss: 0.762211263179779
Epoch: 0, Loss: 0.7228502631187439
Epoch: 0, Loss: 0.699448823928833
Epoch: 0, Loss: 0.66012620

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.7679634094238281
Epoch: 0, Loss: 0.9636171460151672
Epoch: 0, Loss: 0.9866716861724854
Epoch: 0, Loss: 0.6101205348968506
Epoch: 0, Loss: 0.7011028528213501
Epoch: 0, Loss: 0.9441792368888855
Epoch: 0, Loss: 0.7488245368003845
Epoch: 0, Loss: 0.6321273446083069
Epoch: 0, Loss: 0.6848502159118652
Epoch: 0, Loss: 0.6269491910934448
Epoch: 0, Loss: 0.7505160570144653
Epoch: 0, Loss: 0.5807337164878845
Epoch: 0, Loss: 0.7787795662879944
Epoch: 0, Loss: 1.0394244194030762
Epoch: 0, Loss: 0.6578269004821777
Epoch: 0, Loss: 0.8629485964775085
Epoch: 0, Loss: 1.1191225051879883
Epoch: 0, Loss: 0.6874159574508667
Epoch: 0, Loss: 0.8006812334060669
Epoch: 0, Loss: 1.0858304500579834
Epoch: 0, Loss: 0.5017248392105103
Epoch: 0, Loss: 0.7762154936790466
Epoch: 0, Loss: 0.5981845259666443
Epoch: 0, Loss: 0.711452305316925
Epoch: 0, Loss: 0.6672523021697998
Epoch: 0, Loss: 0.6684299111366272
Epoch: 0, Loss: 0.5522527694702148
Epoch: 0, Loss: 0.6930052638053894
Epoch: 0, Loss: 0.879

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.6962562799453735
Epoch: 0, Loss: 0.6901085376739502
Epoch: 0, Loss: 0.7146985530853271
Epoch: 0, Loss: 0.5802930593490601
Epoch: 0, Loss: 0.8511291742324829
Epoch: 0, Loss: 0.8371264934539795
Epoch: 0, Loss: 0.8078445196151733
Epoch: 0, Loss: 0.6102094650268555
Epoch: 0, Loss: 0.6314709782600403
Epoch: 0, Loss: 0.8492393493652344
Epoch: 0, Loss: 0.6624542474746704
Epoch: 0, Loss: 0.708987832069397
Epoch: 0, Loss: 0.7613657712936401
Epoch: 0, Loss: 0.6175095438957214
Epoch: 0, Loss: 0.6577478051185608
Epoch: 0, Loss: 0.7521757483482361
Epoch: 0, Loss: 0.6228294372558594
Epoch: 0, Loss: 0.737365186214447
Epoch: 0, Loss: 0.5938457250595093
Epoch: 0, Loss: 0.5625768303871155
Epoch: 0, Loss: 0.680483341217041
Epoch: 0, Loss: 0.6937091946601868
Epoch: 0, Loss: 0.8319182395935059
Epoch: 0, Loss: 0.6883761286735535
Epoch: 0, Loss: 0.7839921712875366
Epoch: 0, Loss: 0.6769206523895264
Epoch: 0, Loss: 0.771885335445404
Epoch: 0, Loss: 0.6415337324142456
Epoch: 0, Loss: 0.675129

In [10]:
# model_base = 'bert-base-uncased'

# from transformers import BertTokenizer as tkz
# from transformers import BertForSequenceClassification as mpc

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

overall_accuracies, accuracies, f1s, precisions, recalls = [], [], [], [], []

# for run in range(1):
for run in range(3):
  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  print(model_path)

  model = mpc.from_pretrained(model_base, num_labels = 2)
  model.load_state_dict(torch.load(model_path))

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  tokenizer = tkz.from_pretrained(model_base)
  validation_dataset = CladderDataset(validation_data, tokenizer)
  validation_dataloader = DataLoader(validation_dataset, batch_size=8)




  model.eval()
  correct_predictions = 0
  total_predictions = 0

  accuracy, f1, precision, recall = [], [], [], []
  batches = 0

  def compute_metrics(y_true, y_pred):
      accuracy = accuracy_score(y_true, y_pred)
      # print(y_true)
      # print(y_pred)
      f1 = f1_score(y_true, y_pred)

      precision = precision_score(y_true, y_pred)
      # print(precision)
      recall = recall_score(y_true, y_pred)
      return [accuracy, f1, precision, recall]

  with torch.no_grad():
      for batch in validation_dataloader:
          input_ids = batch['input_ids'].squeeze(1)
          attention_mask = batch['attention_mask'].squeeze(1)
          labels = batch['labels'].cpu()  # ground truth
          # print(labels)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          # print(outputs)
          # print(outputs.logits)
          predictions = torch.argmax(outputs.logits, dim=1).cpu()  # predictions
          # print(predictions)

          calculated_metrics = compute_metrics(labels, predictions)
          accuracy.append(calculated_metrics[0])
          f1.append(calculated_metrics[1])
          precision.append(calculated_metrics[2])
          recall.append(calculated_metrics[3])
          batches += 1

          correct_predictions += (predictions == labels).sum().item()
          total_predictions += labels.size(0)

  overall_accuracy = correct_predictions / total_predictions
  print(f"Overall Accuracy: {overall_accuracy:.4f}")
  overall_accuracies.append(overall_accuracy)

  accuracy_var = sum(accuracy)/batches
  print(f"Accuracy: {sum(accuracy)/batches:.4f}")
  print(accuracy_var)
  accuracies.append(accuracy_var)

  f1_var = sum(f1)/batches
  print(f"F1: {sum(f1)/batches:.4f}")
  print(f1_var)
  f1s.append(f1_var)

  precision_var = sum(precision)/batches
  print(f"Precision: {sum(precision)/batches:.4f}")
  print(precision_var)
  precisions.append(precision_var)

  recall_var = sum(recall)/batches
  print(f"Recall: {sum(recall)/batches:.4f}")
  print(recall_var)
  recalls.append(precision_var)

  print(f'Number of batches: {batches}')

cladder_bert-base-uncased_0.pt


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-def

Overall Accuracy: 0.5225
Accuracy: 0.5225
0.5225474683544303
F1: 0.4184
0.41841562094726675
Precision: 0.5048
0.5047995780590717
Recall: 0.4112
0.4111512959614228
Number of batches: 316
cladder_bert-base-uncased_1.pt


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Overall Accuracy: 0.5704
Accuracy: 0.5704
0.5704113924050633
F1: 0.5215
0.5215010552035872
Precision: 0.5697
0.5697106690777576
Recall: 0.5503
0.5503164556962026
Number of batches: 316
cladder_bert-base-uncased_2.pt


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Overall Accuracy: 0.4996
Accuracy: 0.4996
0.49960443037974683
F1: 0.5180
0.5180019699007044
Precision: 0.4973
0.4973176612417118
Recall: 0.6125
0.6124623267028328
Number of batches: 316


In [ ]:
overall_accuracies

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        print(f"predictions: {predictions}, labels: {labels}")
        total_predictions += labels.size(0)

print(f"total_predictions: {total_predictions}")

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")


In [ ]:
# Check by removing "Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships:"


In [ ]:
`from datasets import load_dataset

dataset = load_dataset("causalnlp/CLadder")

In [ ]:
dataset = dataset['train'].to_pandas()

In [ ]:
dataset['prompt'] = dataset.apply(lambda row: (row['prompt']).removeprefix('Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: '), axis=1)




In [ ]:
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['ground_truth']=='yes' else 0, axis=1)

dataset = json.loads(dataset.to_json(orient ='records'))

# Shuffle the dataset
random.shuffle(dataset)

# Define split sizes for a 75-25 split
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size

# Split the dataset
train_data = dataset[:train_size]
validation_data = dataset[train_size:]

In [ ]:
# Prepare the DataLoader
cladder_dataset = CladderDataset(train_data, tokenizer)
dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# Training Loop
model.train()

for epoch in range(3):  # Number of epochs
  for batch in dataloader:
      optimizer.zero_grad()

      input_ids = batch['input_ids'].squeeze(1)
      attention_mask = batch['attention_mask'].squeeze(1)
      labels = batch['labels']

      outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss

      loss.backward()
      optimizer.step()

      print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [ ]:
validation_dataset = CladderDataset(validation_data, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        print(f"predictions: {predictions}, labels: {labels}")
        total_predictions += labels.size(0)

print(f"total_predictions: {total_predictions}")

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")
